In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
from owslib.wfs import WebFeatureService
from dotenv import load_dotenv

# Create a spark session
spark = (
    SparkSession.builder.appName("analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "6g")
    .getOrCreate()
)

22/09/19 22:16:38 WARN Utils: Your hostname, AryansLaptop resolves to a loopback address: 127.0.1.1; using 172.28.41.58 instead (on interface eth0)
22/09/19 22:16:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/19 22:16:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Combining datasets

In [2]:
sdf = spark.read.parquet("../data/curated/mergedf.parquet/")
print(sdf.count())
sdf.limit(5)

13614675


merchant_abn,consumer_id,user_id,dollar_value,order_id,order_datetime,customer_name,address,state,postcode,gender,company_name,tags,take_rate,revenue_band
69666829657,226,5162,79.65159982605903,8765ef9f-dba6-407...,2021-08-20,Courtney Torres,6945 Higgins Brooks,SA,5581,Female,Sem Ut Institute,bicycle shops - s...,2.86,c
50321300271,226,5162,594.2915496790856,79f2842d-f8b2-4fd...,2022-06-01,Courtney Torres,6945 Higgins Brooks,SA,5581,Female,Augue Industries,bicycle shops - s...,4.24,b
70610974780,226,5162,52.820359204536665,bdf345c8-4c5d-48e...,2021-08-20,Courtney Torres,6945 Higgins Brooks,SA,5581,Female,Elit Aliquam PC,tent and awning s...,6.93,a
17324645993,226,5162,27.12729568273566,0a44d623-e325-4fc...,2022-05-12,Courtney Torres,6945 Higgins Brooks,SA,5581,Female,Eget Metus In Cor...,tent and awning s...,5.73,a
15582655078,226,5162,75.31904078962366,47ddf8e0-5f72-408...,2021-08-21,Courtney Torres,6945 Higgins Brooks,SA,5581,Female,Eu Odio Phasellus...,"gift, card, novel...",6.77,a


In [3]:
postcodedf = spark.read.parquet("../data/curated/postcodedf.parquet/")
print(postcodedf.count())
postcodedf.limit(5)

5492


postcode,sa2_code
1008,117031337
1150,117031337
2100,122031429
2200,119011571
2338,110041201


In [4]:
pop_sdf = spark.read.parquet("../data/curated/pop_sdf.parquet/")
print(pop_sdf.count())
populationdf = pop_sdf.withColumnRenamed("SA2 code","sa2_code")
populationdf.limit(5)

2450


State/Terr,sa2_code,SA2 name,Total,Under 10,Adolescent,Young adult,Middle age,Old
New South Wales,121011686,Lane Cove,15931,2246,1683,3108,5838,3056
New South Wales,121011687,Willoughby,13024,1829,1862,1679,4949,2705
New South Wales,121021403,Asquith - Mount C...,22134,2949,3092,3674,8221,4198
New South Wales,121021404,Berowra - Brookly...,11793,1513,1805,1444,4385,2646
New South Wales,121021406,Normanhurst - Tho...,19199,2395,2868,2589,6962,4385


In [5]:
incomedf = spark.read.parquet("../data/curated/incomedf.parquet/")
print(incomedf.count())
incomedf.limit(5)

2239


sa2_code,num_earners,median_age,median_income,mean_income
213011340,11238,46,51181,61177
213021341,7942,43,57585,69301
213021344,11216,41,66161,80219
213021345,3015,41,52078,60249
213021346,9996,47,63892,90668


In [6]:
consumerfrauddf = spark.read.parquet("../data/curated/consumerfrauddf.parquet/")
print(consumerfrauddf.count())
consumerfrauddf = consumerfrauddf.withColumnRenamed("fraud_probability","consumer_fraud_%")
consumerfrauddf.limit(5)

34765


user_id,order_datetime,consumer_fraud_%
3753,2022-02-16,48.85325253622543
9646,2021-09-23,47.83931206340956
243,2021-09-02,50.88971939168309
3907,2021-10-07,38.58123424858352
14864,2021-11-29,27.072321329372105


In [7]:
merchantfrauddf = spark.read.parquet("../data/curated/merchantfrauddf.parquet/")
print(merchantfrauddf.count())
merchantfrauddf = merchantfrauddf.withColumnRenamed("fraud_probability","merchant_fraud_%")
merchantfrauddf.limit(5)

114


merchant_abn,order_datetime,merchant_fraud_%
19492220327,2021-11-28,44.403658647495355
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


External dataset joins

In [8]:
print(sdf.count())
finaldf = sdf.join(postcodedf, "postcode")
print(finaldf.count())
finaldf = finaldf.join(populationdf, "sa2_code")
print(finaldf.count())
finaldf = finaldf.join(incomedf, "sa2_code")
print(finaldf.count())
finaldf.limit(5)

13614675


23253251


20952106


20774556
22/09/19 22:16:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


sa2_code,postcode,merchant_abn,consumer_id,user_id,dollar_value,order_id,order_datetime,customer_name,address,state,gender,company_name,tags,take_rate,revenue_band,State/Terr,SA2 name,Total,Under 10,Adolescent,Young adult,Middle age,Old,num_earners,median_age,median_income,mean_income
405041127,5581,69666829657,226,5162,79.65159982605903,8765ef9f-dba6-407...,2021-08-20,Courtney Torres,6945 Higgins Brooks,SA,Female,Sem Ut Institute,bicycle shops - s...,2.86,c,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377
405041127,5581,50321300271,226,5162,594.2915496790856,79f2842d-f8b2-4fd...,2022-06-01,Courtney Torres,6945 Higgins Brooks,SA,Female,Augue Industries,bicycle shops - s...,4.24,b,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377
405041127,5581,70610974780,226,5162,52.820359204536665,bdf345c8-4c5d-48e...,2021-08-20,Courtney Torres,6945 Higgins Brooks,SA,Female,Elit Aliquam PC,tent and awning s...,6.93,a,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377
405041127,5581,17324645993,226,5162,27.12729568273566,0a44d623-e325-4fc...,2022-05-12,Courtney Torres,6945 Higgins Brooks,SA,Female,Eget Metus In Cor...,tent and awning s...,5.73,a,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377
405041127,5581,15582655078,226,5162,75.31904078962366,47ddf8e0-5f72-408...,2021-08-21,Courtney Torres,6945 Higgins Brooks,SA,Female,Eu Odio Phasellus...,"gift, card, novel...",6.77,a,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377


Fraud joins

In [9]:
print(finaldf.count())
finaldf = finaldf.join(consumerfrauddf, ["order_datetime", "user_id"], "leftouter")
print(finaldf.count())
finaldf = finaldf.join(merchantfrauddf, ["order_datetime", "merchant_abn"], "leftouter")
print(finaldf.count())
finaldf.limit(5)

20774556


20774556


20774556


order_datetime,merchant_abn,user_id,sa2_code,postcode,consumer_id,dollar_value,order_id,customer_name,address,state,gender,company_name,tags,take_rate,revenue_band,State/Terr,SA2 name,Total,Under 10,Adolescent,Young adult,Middle age,Old,num_earners,median_age,median_income,mean_income,consumer_fraud_%,merchant_fraud_%
2021-08-20,69666829657,5162,405041127,5581,226,79.65159982605903,8765ef9f-dba6-407...,Courtney Torres,6945 Higgins Brooks,SA,Female,Sem Ut Institute,bicycle shops - s...,2.86,c,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,null,null
2022-06-01,50321300271,5162,405041127,5581,226,594.2915496790856,79f2842d-f8b2-4fd...,Courtney Torres,6945 Higgins Brooks,SA,Female,Augue Industries,bicycle shops - s...,4.24,b,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,null,null
2021-08-20,70610974780,5162,405041127,5581,226,52.820359204536665,bdf345c8-4c5d-48e...,Courtney Torres,6945 Higgins Brooks,SA,Female,Elit Aliquam PC,tent and awning s...,6.93,a,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,null,null
2022-05-12,17324645993,5162,405041127,5581,226,27.12729568273566,0a44d623-e325-4fc...,Courtney Torres,6945 Higgins Brooks,SA,Female,Eget Metus In Cor...,tent and awning s...,5.73,a,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,null,null
2021-08-21,15582655078,5162,405041127,5581,226,75.31904078962366,47ddf8e0-5f72-408...,Courtney Torres,6945 Higgins Brooks,SA,Female,Eu Odio Phasellus...,"gift, card, novel...",6.77,a,South Australia,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,null,null


In [10]:
finaldf = finaldf.drop("customer_name","address","State/Terr")
finaldf = finaldf.na.fill(0.1)
finaldf.limit(5)

order_datetime,merchant_abn,user_id,sa2_code,postcode,consumer_id,dollar_value,order_id,state,gender,company_name,tags,take_rate,revenue_band,SA2 name,Total,Under 10,Adolescent,Young adult,Middle age,Old,num_earners,median_age,median_income,mean_income,consumer_fraud_%,merchant_fraud_%
2021-08-20,69666829657,5162,405041127,5581,226,79.65159982605903,8765ef9f-dba6-407...,SA,Female,Sem Ut Institute,bicycle shops - s...,2.86,c,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1
2022-06-01,50321300271,5162,405041127,5581,226,594.2915496790856,79f2842d-f8b2-4fd...,SA,Female,Augue Industries,bicycle shops - s...,4.24,b,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1
2021-08-20,70610974780,5162,405041127,5581,226,52.820359204536665,bdf345c8-4c5d-48e...,SA,Female,Elit Aliquam PC,tent and awning s...,6.93,a,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1
2022-05-12,17324645993,5162,405041127,5581,226,27.12729568273566,0a44d623-e325-4fc...,SA,Female,Eget Metus In Cor...,tent and awning s...,5.73,a,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1
2021-08-21,15582655078,5162,405041127,5581,226,75.31904078962366,47ddf8e0-5f72-408...,SA,Female,Eu Odio Phasellus...,"gift, card, novel...",6.77,a,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1


## Dealing with current fraud data

In [11]:
MERCHANT_THRESH = 20
CONSUMER_THRESH = 20

print(finaldf.count())
testdf = finaldf.filter(finaldf["merchant_fraud_%"] < MERCHANT_THRESH)
testdf = finaldf.filter(finaldf["consumer_fraud_%"] < CONSUMER_THRESH)
print(testdf.count(), finaldf.count()-testdf.count())

20774556


20759042 15514


Keeping the fraud prob below 20% for each give us a good chance that most fraud data has been removed <br>
The removal of 15 thousand records is not alot considering we still have 20 million plus records still left <br>
The fact that most of the fraud transactions are now removed allows us to build a metric for determining future fraud

In [12]:
finaldf = finaldf.filter(finaldf["merchant_fraud_%"] < MERCHANT_THRESH)
finaldf = finaldf.filter(finaldf["consumer_fraud_%"] < CONSUMER_THRESH)

In [13]:
finaldf.limit(5)

order_datetime,merchant_abn,user_id,sa2_code,postcode,consumer_id,dollar_value,order_id,state,gender,company_name,tags,take_rate,revenue_band,SA2 name,Total,Under 10,Adolescent,Young adult,Middle age,Old,num_earners,median_age,median_income,mean_income,consumer_fraud_%,merchant_fraud_%
2021-08-20,69666829657,5162,405041127,5581,226,79.65159982605903,8765ef9f-dba6-407...,SA,Female,Sem Ut Institute,bicycle shops - s...,2.86,c,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1
2022-06-01,50321300271,5162,405041127,5581,226,594.2915496790856,79f2842d-f8b2-4fd...,SA,Female,Augue Industries,bicycle shops - s...,4.24,b,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1
2021-08-20,70610974780,5162,405041127,5581,226,52.820359204536665,bdf345c8-4c5d-48e...,SA,Female,Elit Aliquam PC,tent and awning s...,6.93,a,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1
2022-05-12,17324645993,5162,405041127,5581,226,27.12729568273566,0a44d623-e325-4fc...,SA,Female,Eget Metus In Cor...,tent and awning s...,5.73,a,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1
2021-08-21,15582655078,5162,405041127,5581,226,75.31904078962366,47ddf8e0-5f72-408...,SA,Female,Eu Odio Phasellus...,"gift, card, novel...",6.77,a,Yorke Peninsula -...,7621,691,681,818,2022,3409,3438,53,40165,52377,0.1,0.1


## Dealing with future fraud

2 main ways to determine fraud:
- High number of transactions in a given day
- High transaction amounts

### Merchant

In [14]:
# this gets the stats for each merchant for each day
merchanttestdf = finaldf.groupBy("merchant_abn","order_datetime").agg(
    mean("dollar_value").alias("mean_amount"),
    min("dollar_value").alias("min_amount"),
    max("dollar_value").alias("max_amount"),
    count("dollar_value").alias("transactions"),
    ).orderBy(desc("transactions"))

merchanttestdf.limit(5)

merchant_abn,order_datetime,mean_amount,min_amount,max_amount,transactions
24852446429,2021-11-26,30.46417361981634,0.4388815965837197,168.92668885555742,1916
24852446429,2021-11-27,31.636556215187312,1.003369617502435,193.03651325623903,1783
86578477987,2021-11-26,33.8695396597357,0.1615814452824192,163.1881979840336,1779
24852446429,2021-11-29,30.50184795665986,0.7022808404239752,137.595051098878,1744
49891706470,2021-11-26,28.682157995166218,3.001709666041197,54.95671169593744,1701


In [15]:
# this gets the average stats for a merchant on any given day
merchanttestdf = merchanttestdf.groupBy("merchant_abn").agg(
    mean("mean_amount").alias("mean_amount"),
    mean("min_amount").alias("min_amount"),
    mean("max_amount").alias("max_amount"),
    mean("transactions").alias("mean_transactions"),
    min("transactions").alias("min_transactions"),
    max("transactions").alias("max_transactions"),
    ).orderBy(desc("max_transactions"))

merchanttestdf.limit(5)

merchant_abn,mean_amount,min_amount,max_amount,mean_transactions,min_transactions,max_transactions
24852446429,29.994179500150423,1.0013482034321513,132.63726075232657,727.2062706270627,442,1916
86578477987,34.97995533972651,1.1720818299391733,153.98697166956993,685.6435643564356,412,1779
49891706470,28.94237492419655,3.1556904183096153,54.84419033696536,621.6039603960396,335,1701
64203420245,28.95612306014824,3.14451491936713,54.85378248897557,656.2095709570957,360,1681
46804135891,30.035261029384944,0.0972265437192253,190.93127738260756,588.4306930693069,331,1607


We now have a dataset that gives us all the statistics we need to determine if future merchant transactions are fraud <br>
for the time being if a merchant has > 1.5 x max_transactions in a day it is considered fraud <br>
if a merchant has > 1.5 x max_amount in a day it is considered fraud

### User

In [16]:
# this gets the stats for each user for each day
usertestdf = finaldf.groupBy("user_id","order_datetime").agg(
    mean("dollar_value").alias("mean_amount"),
    min("dollar_value").alias("min_amount"),
    max("dollar_value").alias("max_amount"),
    count("dollar_value").alias("transactions"),
    ).orderBy(desc("transactions"))

usertestdf.limit(5)

22/09/19 22:17:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:39 WARN RowBasedKeyValueBatch: Calling spill() on

22/09/19 22:17:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:44 WARN RowBasedKeyValueBatch: Calling spill() on

user_id,order_datetime,mean_amount,min_amount,max_amount,transactions
20931,2021-11-29,119.06473250477067,28.04893125562283,445.0802574763232,104
17417,2021-11-26,72.40206052677397,4.4464593786411095,157.8166304195111,98
6072,2021-11-22,269.3998632737218,6.712747985289963,622.847977814607,98
17417,2021-11-29,167.94750291512926,1.6711964900188083,719.3382556816264,98
20651,2021-11-25,62.674804005227486,13.418453836985401,125.3852608556601,98


In [17]:
# this gets the average stats for a user on any given day
usertestdf = usertestdf.groupBy("user_id").agg(
    mean("mean_amount").alias("mean_amount"),
    mean("min_amount").alias("min_amount"),
    mean("max_amount").alias("max_amount"),
    mean("transactions").alias("mean_transactions"),
    min("transactions").alias("min_transactions"),
    max("transactions").alias("max_transactions"),
    ).orderBy(desc("max_transactions"))

usertestdf.limit(5)

22/09/19 22:17:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/09/19 22:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/19 22:17:54 WARN RowBasedKeyValueBatch: Calling spill() on

user_id,mean_amount,min_amount,max_amount,mean_transactions,min_transactions,max_transactions
20931,163.86220255628083,125.76545989482213,208.98524908238096,20.107142857142858,13,104
20651,157.1074921825416,105.06627100387666,219.10105072162514,21.0,14,98
6072,159.56756149497772,101.95187779113559,224.5209349892101,21.899441340782122,14,98
17417,147.8868588470973,104.75892839781038,196.83524356068264,21.517808219178082,14,98
13545,158.1379638827785,121.45901194379616,205.68937045590945,20.06830601092896,13,91


We now have a dataset that gives us all the statistics we need to determine if future user transactions are fraud <br>
for the time being if a user has > 1.5 x max_transactions in a day it is considered fraud <br>
if a user has > 1.5 x max_amount in a day it is considered fraud

Now remains the case where a user or merchant has little to no data to get these metrics from <br>
We could either set our own thresholds based on logical reasoning for fraud detection<br>
Or we could look at the distributions of existing data to determine thresholds for fraud detection